In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
import numpy as np
from Models import MoELSTM
import os
from collections import OrderedDict
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader

from typing import List, Tuple, Optional, Dict
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
import random
from Models import MoELSTM, LSTMModel, train_model
from Preprocess import (
    compute_metrics,
    convert_timeseries_to_numpy,
    create_dataloader,
    load_building_series,
    split_series_list,
)
from Models import model_fn
from tqdm import tqdm
from my_utils import train_model, load_energy_data_feather, get_weights, set_weights
from energy_ts_diffusion.task import convert_timeseries_to_numpy  # adjust as per your project
from tqdm import tqdm


In [2]:


# Config
# List of models to experiment with
MODEL_NAMES = ["lstm", "gru"]

# Config
NUM_CLIENTS = 1410
CLIENT_FRAC = 0.15
NUM_ROUNDS = 50
LOCAL_EPOCHS = 5
LR = 0.001
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DATA_FILE ="train_final.feather" # "meter_0_data_cleaned.feather"


In [3]:
train_loader, _ = load_energy_data_feather(403, filepath=DATA_FILE)

In [ ]:

@torch.no_grad()
def rolling_forecast_on_test(cid, model, filepath="meter_"
""
"", input_len=24, output_len=8):
    """
    Perform rolling window forecast on the test data using a trained model and return
    unscaled predictions and ground truths with actual timestamps.

    Args:
        cid (int): Client/building ID.
        model (nn.Module): Trained PyTorch model.
        filepath (str): Path to the Feather file.
        input_len (int): Input sequence length.
        output_len (int): Prediction horizon.

    Returns:
        Tuple[List[TimeSeries], List[TimeSeries]]: (predictions_ts_list, ground_truth_ts_list)
    """

    # Load and filter data
    df = pd.read_feather(filepath)
    df = df[df['building_id'] == cid]
    df['meter_reading'] = df['meter_reading'].fillna(0)

    if df.empty:
        raise ValueError(f"No data found for building_id {cid}")

    # Create TimeSeries and scale
    ts = TimeSeries.from_dataframe(
        df,
        time_col='timestamp',
        value_cols='meter_reading',
        fill_missing_dates=True,
        freq='h'
    )

    _, test_series = ts.split_before(0.75)

    # Scale
    scaler = MinMaxScaler(feature_range=(0.1, 1))
    transformer = Scaler(scaler)
    test_series_scaled = transformer.fit_transform(test_series)

    test_values_scaled = test_series_scaled.values().squeeze()
    test_timestamps = test_series_scaled.time_index

    predictions_ts_list = []
    ground_truth_ts_list = []

    model.eval()
    device = next(model.parameters()).device

    for i in range(0, len(test_values_scaled) - input_len - output_len + 1):
        input_seq = test_values_scaled[i:i+input_len]
        true_output = test_values_scaled[i+input_len:i+input_len+output_len]
        true_time = test_timestamps[i+input_len:i+input_len+output_len]

        input_tensor = torch.tensor(input_seq, dtype=torch.float32).unsqueeze(0).unsqueeze(-1).to(device)  # [1, input_len, 1]

        pred = model(input_tensor)
        if pred.dim() == 3:
            pred = pred.squeeze(0).squeeze(-1)
        else:
            pred = pred.squeeze(0)

        # Convert prediction & ground truth to TimeSeries
        pred_ts = TimeSeries.from_times_and_values(true_time, pred.cpu().numpy())
        true_ts = TimeSeries.from_times_and_values(true_time, true_output)

        # Inverse transform
        pred_unscaled = transformer.inverse_transform(pred_ts)
        true_unscaled = transformer.inverse_transform(true_ts)

        predictions_ts_list.append(pred_unscaled)
        ground_truth_ts_list.append(true_unscaled)

    return predictions_ts_list, ground_truth_ts_list


In [3]:

def get_model_predictions_csv(model_name: str, cid: int,aggr_strat: str ,rounds: list, model_dir: str, output_csv: str):
    """
    For each round, load the model, predict on test set for cid, and save all preds in a single CSV.
    """
    rows = []

    for rnd in tqdm(rounds):
        model_path = os.path.join(model_dir, f"{model_name}_round_{rnd}_{aggr_strat}.pt")

        if not os.path.exists(model_path):
            print(f"[WARN] Model not found: {model_path}")
            continue

        model = model_fn(model_name)
        model.load_state_dict(torch.load(model_path, map_location="cpu"))
        model.eval()

        pred_ts_list, gt_ts_list = rolling_forecast_on_test(cid=cid, model=model)

        for pred_ts, true_ts in zip(pred_ts_list, gt_ts_list):
            df_pred = pd.DataFrame({"timestamp": pred_ts.time_index, "pred": pred_ts.values().squeeze()})
            df_true = pd.DataFrame({"timestamp": true_ts.time_index, "true": true_ts.values().squeeze()})

            # df_merged = df_true.join(df_pred, how="inner")
            df_merged = pd.merge(df_true, df_pred, on="timestamp", how="inner")
            df_merged["round"] = rnd


            rows.append(df_merged[["timestamp", "true", "pred", "round"]])

    # Combine all rows
    final_df = pd.concat(rows, ignore_index=True)
    final_df.to_csv(output_csv, index=False)
    print(f"[INFO] Forecasts written to {output_csv}")



## for each building id store pred - cid

In [4]:

# def get_model_predictions_csv_building(model_name: str, cid: int,aggr_strat: str ,rounds: list, model_dir: str, output_csv: str):
#     """
#     For each round, load the model, predict on test set for cid, and save all preds in a single CSV.
#     """
#     rows = []

#     for rnd in tqdm(rounds):
#         model_path = os.path.join(model_dir, f"{model_name}_round_{rnd}_{aggr_strat}.pt")

#         if not os.path.exists(model_path):
#             print(f"[WARN] Model not found: {model_path}")
#             continue

#         model = model_fn(model_name)
#         model.load_state_dict(torch.load(model_path, map_location="cpu"))
#         model.eval()

#         pred_ts_list, gt_ts_list = rolling_forecast_on_test(cid=cid, model=model)

#         for pred_ts, true_ts in zip(pred_ts_list, gt_ts_list):
#             df_pred = pd.DataFrame({"timestamp": pred_ts.time_index, "pred": pred_ts.values().squeeze()})
#             df_true = pd.DataFrame({"timestamp": true_ts.time_index, "true": true_ts.values().squeeze()})

#             # df_merged = df_true.join(df_pred, how="inner")
#             df_merged = pd.merge(df_true, df_pred, on="timestamp", how="inner")
#             df_merged["round"] = rnd


#             rows.append(df_merged[["timestamp", "true", "pred", "round"]])

#     # Combine all rows
#     final_df = pd.concat(rows, ignore_index=True)
#     final_df.to_csv(output_csv, index=False)
#     print(f"[INFO] Forecasts written to {output_csv}")



In [5]:
# get_model_predictions_csv(
#     model_name="gru",
#     cid=42,
#     rounds=[1, 2, 3, 4, 5],
#     model_dir="results/gru",
#     output_csv="gru_rolling_predictions.csv"
# )


In [6]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

def smape(y_true, y_pred):
    """Symmetric Mean Absolute Percentage Error."""
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    return np.mean(np.where(denominator == 0, 0, np.abs(y_true - y_pred) / denominator)) * 100

def mape(y_true, y_pred):
    """Mean Absolute Percentage Error."""
    y_true = np.where(y_true == 0, 1e-8, y_true)  # avoid division by zero
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def evaluate_forecast_metrics_per_round(csv_path):
    """
    Reads forecast CSV and computes MAPE, MAE, SMAPE, RMSE, and MSE per round.

    Args:
        csv_path (str): Path to the CSV with columns: timestamp, true, pred, round

    Returns:
        pd.DataFrame: Metrics summary per round
    """
    df = pd.read_csv(csv_path)
    if df.empty:
        raise ValueError("CSV is empty or invalid")

    metrics_list = []

    for rnd in sorted(df['round'].unique()):
        df_rnd = df[df['round'] == rnd]
        y_true = df_rnd["true"].values
        y_pred = df_rnd["pred"].values

        mae = mean_absolute_error(y_true, y_pred)
        mse = mean_squared_error(y_true, y_pred)
        rmse = np.sqrt(mse)
        mape_val = mape(y_true, y_pred)
        smape_val = smape(y_true, y_pred)

        metrics_list.append({
            "round": rnd,
            "MAE": mae,
            "MSE": mse,
            "RMSE": rmse,
            "MAPE (%)": mape_val,
            "SMAPE (%)": smape_val
        })

    metrics_df = pd.DataFrame(metrics_list)
    return metrics_df


In [7]:
MODEL_NAME = "gru"
CID = 45
ROUND = [1,2,3,4,5,6,7,8,9]
MODEL_DIR = f"results/{MODEL_NAME}"
STRATEGY = "kr"
OUTPUT_DIR = f"predictions/{CID}_{MODEL_NAME}_{STRATEGY}.csv"

In [9]:
get_model_predictions_csv(
    model_name=MODEL_NAME,
    cid=CID,
    rounds=ROUND,
    model_dir=MODEL_DIR,
    output_csv=OUTPUT_DIR,
    aggr_strat= STRATEGY
)

  0%|          | 0/9 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'meter_0_data_cleaned.feather'

In [9]:
metrics_df = evaluate_forecast_metrics_per_round(OUTPUT_DIR)
print(metrics_df.to_string(index=False))


 round      MAE       MSE     RMSE   MAPE (%)  SMAPE (%)
     1 1.923647  4.500009 2.121322  69.997375 114.260351
     2 1.255574  2.282222 1.510702  43.709379  60.022051
     3 0.854270  1.111473 1.054264  31.378716  35.242188
     4 2.027142  4.629476 2.151622  76.801186 126.530583
     5 1.852406  4.092056 2.022883  70.587240 110.434803
     6 1.770699  4.543252 2.131491  71.250029  94.049303
     7 4.777556 24.926722 4.992667 191.032033 197.836468
     8 4.177765 24.706141 4.970527 170.923942 155.682856
     9 2.760312 11.990074 3.462669 106.734097 119.880668


In [ ]:
MODEL_NAME = "gru"
CID = 5
ROUND = [1,2,3,4,5,6,7,8,9]
MODEL_DIR = f"results/{MODEL_NAME}"
STRATEGY = "fedAvg"
OUTPUT_DIR = f"predictions/{CID}_{MODEL_NAME}_{STRATEGY}.csv"

get_model_predictions_csv(
    model_name=MODEL_NAME,
    cid=CID,
    rounds=ROUND,
    model_dir=MODEL_DIR,
    output_csv=OUTPUT_DIR,
    aggr_strat= STRATEGY
)

metrics_df = evaluate_forecast_metrics_per_round(OUTPUT_DIR)
print(metrics_df.to_string(index=False))

In [ ]:

# Configurations
MODELS = ["gru", "lstm", "moe_gru", "moe_lstm"]
# STRATEGIES = ["fedAvg", "fedAvgM", "kr_norm", "kr", "kr_sft"]
STRATEGIES = ["fedAvg","fedAvgM","diff", "diff2", "diff_rev", "scaffold"]
# CID = 879
ROUNDS = list(range(7, 11))
BASE_RESULTS_DIR = "results"
BASE_OUTPUT_DIR = "predictions"
METRICS_DIR = "metrics"
# return rolling forecast - prediction - true add this in new csv with building_id 


# Create directories if needed
os.makedirs(BASE_OUTPUT_DIR, exist_ok=True)
os.makedirs(METRICS_DIR, exist_ok=True)
for CID in range(1411):
    for model_name in MODELS:
        for strategy in STRATEGIES:
            model_dir = os.path.join(BASE_RESULTS_DIR, model_name)
            output_csv = os.path.join(BASE_OUTPUT_DIR, f"{CID}_{model_name}_{strategy}.csv")
            metrics_csv = os.path.join(METRICS_DIR, f"cid{CID}_{model_name}_{strategy}_metrics.csv")

            print(f"\n▶ Model: {model_name}, Strategy: {strategy}")

            try:
                # Run prediction and save CSV
                get_model_predictions_csv(
                    model_name=model_name,
                    cid=CID,
                    rounds=ROUNDS,
                    model_dir=model_dir,
                    output_csv=output_csv,
                    aggr_strat=strategy
                )

                # Compute and save metrics
                metrics_df = evaluate_forecast_metrics_per_round(output_csv)
                metrics_df.to_csv(metrics_csv, index=False)

                print(f"Metrics saved to {metrics_csv}")

            except Exception as e:
                print(f"[ERROR] model={model_name}, strategy={strategy}: {e}")



▶ Model: gru, Strategy: fedAvg


 11%|█         | 1/9 [00:11<01:34, 11.86s/it]


KeyboardInterrupt: 